## Kindle Review sentiment Analysis

## Best Practices:
1. Preprocessing and clening
2. train test split
3. Bow,tfidf,word2vec
4. train ml algorithms

In [1]:
import pandas as pd

In [2]:
#Load the data
df = pd.read_csv('kindle.csv')

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [4]:
df = df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [5]:
df.shape

(12000, 2)

In [6]:
df.isna().sum()

reviewText    0
rating        0
dtype: int64

In [7]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [8]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [9]:
## Preprocessing and Clenaning

In [10]:
#Positive Review is 1 and Negative review is 0

df['rating']=df['rating'].apply(lambda x : 0 if x<3 else 1)

In [11]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [12]:
df['rating'].unique()

array([1, 0], dtype=int64)

In [13]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [14]:
##1. lower all the cases
df['reviewText']=df['reviewText'].str.lower()

In [15]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [16]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to C:\Users\Sandeep
[nltk_data]     Bandi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
## Removing special characters
df['reviewText'] = df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9]+','',x))
##Removing Stopwords
df['reviewText'] = df['reviewText'].apply(lambda x:' '.join([y for y in x.split() if y not in stopwords.words('english')]))
#removing url from 
df['reviewText'] = df['reviewText'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-] *[\w@?^=%&/~+#-])?','',str(x)))
#Removing html tags
df['reviewText'] = df['reviewText'].apply(lambda x:BeautifulSoup(x,'lxml').get_text())
#Removing additional spaces
df['reviewText'] = df['reviewText'].apply(lambda x:" ".join(x.split()))

In [18]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [19]:
#Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [20]:
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split() ])

In [21]:
df['reviewText'] = df['reviewText'].apply(lambda x:lemmatize_words(x))

In [22]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [23]:
# train test split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df['reviewText'],df['rating'],test_size=0.2,random_state=42)

In [27]:
#bag of Words
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()

X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [28]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [29]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [30]:
X_train_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf,y_train)

In [32]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [33]:
y_pred_bow = nb_model_bow.predict(X_test_bow)

In [34]:
y_pred_tfidf = nb_model_bow.predict(X_test_tfidf)


In [35]:
print('BOW Score',accuracy_score(y_test,y_pred_bow))
print('TFIDF Score',accuracy_score(y_test,y_pred_tfidf))

BOW Score 0.5775
TFIDF Score 0.57375


In [36]:
confusion_matrix(y_test,y_pred_bow)

array([[499, 304],
       [710, 887]], dtype=int64)

In [37]:
print(classification_report(y_test,y_pred_bow))

              precision    recall  f1-score   support

           0       0.41      0.62      0.50       803
           1       0.74      0.56      0.64      1597

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.63      0.58      0.59      2400



In [38]:
## Assignment use Word2vec coz its huge data

In [39]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [41]:
model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4, sg=1)


In [ ]:
model.pred